In [1]:
import pandas as pd
import numpy as np

In [2]:
ls

Untitled.ipynb         items.csv*             shops.csv*
Untitled1.ipynb        sales_train.csv*       test.csv*
item_categories.csv*   sample_submission.csv*


In [3]:
df_train = pd.read_csv('sales_train.csv')
df_test = pd.read_csv('test.csv')
df_items = pd.read_csv('items.csv')
df_cats = pd.read_csv('item_categories.csv')
df_shops = pd.read_csv('shops.csv')

In [4]:
df_train.head()


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [5]:
df_test.head()


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [6]:
df_items.head()


,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [7]:
df_cats.head()


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [8]:
df_shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [9]:
df_train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [10]:
from sklearn import *

In [11]:
import nltk,datetime

In [12]:
df_train.shape

(2935849, 6)

In [13]:
df_train.dtypes

date               object
date_block_num      int64
shop_id             int64
item_id             int64
item_price        float64
item_cnt_day      float64
dtype: object

In [14]:
df_train['date'] = pd.to_datetime(df_train['date'],format='%d.%m.%Y')


In [15]:
df_train['month']= df_train['date'].dt.month 
df_train['Year'] = df_train['date'].dt.year
df_train= df_train.drop(['date','item_price'],axis=1)

In [16]:
df_train.head(2)

,date_block_num,shop_id,item_id,item_cnt_day,month,Year
0,0,59,22154,1.0,1,2013
1,0,25,2552,1.0,1,2013


In [17]:
df_train = df_train.groupby([c for c in df_train.columns if c not in ['item_cnt_day']], as_index = False)['item_cnt_day'].sum()
df_train = df_train.rename(columns={'item_cnt_day':'item_cnt_month'})

In [18]:
df_train.shape

(1609124, 6)

In [19]:
df_train.head(2)

,date_block_num,shop_id,item_id,month,Year,item_cnt_month
0,0,0,32,1,2013,6.0
1,0,0,33,1,2013,3.0


In [20]:
# find the monthly mean

In [21]:
shop_item_mean = df_train[['shop_id','item_id','item_cnt_month']].groupby(['shop_id','item_id'],as_index = False)[['item_cnt_month']].mean()
shop_item_mean = shop_item_mean.rename(columns = {'item_cnt_month':'item_cnt_mean_month'})

In [22]:
# add mean feture to our dataset

In [23]:
df_train = pd.merge(df_train, shop_item_mean, how = 'left', on = ['shop_id','item_id'])

In [24]:
df_train.head(2)

,date_block_num,shop_id,item_id,month,Year,item_cnt_month,item_cnt_mean_month
0,0,0,32,1,2013,6.0,8.0
1,0,0,33,1,2013,3.0,3.0


In [25]:
df_train.tail(2)

,date_block_num,shop_id,item_id,month,Year,item_cnt_month,item_cnt_mean_month
1609122,33,59,22100,10,2015,1.0,1.000000
1609123,33,59,22102,10,2015,1.0,1.333333


In [26]:
# add last month

In [27]:
shop_prev_month = df_train[df_train['date_block_num']==33][['shop_id','item_id','item_cnt_month']]

In [28]:
shop_prev_month

,shop_id,item_id,item_cnt_month
1577593,2,31,1.0
1577594,2,486,3.0
1577595,2,787,1.0
1577596,2,794,1.0
1577597,2,968,1.0
...,...,...,...
1609119,59,22087,6.0
1609120,59,22088,2.0
1609121,59,22091,1.0
1609122,59,22100,1.0


In [29]:
shop_prev_month = shop_prev_month.rename(columns = {'item_cnt_month':'item_cnt_prev_month'})

In [30]:
shop_prev_month

,shop_id,item_id,item_cnt_prev_month
1577593,2,31,1.0
1577594,2,486,3.0
1577595,2,787,1.0
1577596,2,794,1.0
1577597,2,968,1.0
...,...,...,...
1609119,59,22087,6.0
1609120,59,22088,2.0
1609121,59,22091,1.0
1609122,59,22100,1.0


In [31]:
df_train = pd.merge(df_train,shop_prev_month, how = 'left', on = ['shop_id','item_id'])


In [32]:
df_train.head(2)

,date_block_num,shop_id,item_id,month,Year,item_cnt_month,item_cnt_mean_month,item_cnt_prev_month
0,0,0,32,1,2013,6.0,8.0,NaN
1,0,0,33,1,2013,3.0,3.0,NaN


In [33]:
df_train.head(20)

,date_block_num,shop_id,item_id,month,Year,item_cnt_month,item_cnt_mean_month,item_cnt_prev_month
0,0,0,32,1,2013,6.0,8.0,NaN
1,0,0,33,1,2013,3.0,3.0,NaN
2,0,0,35,1,2013,1.0,7.5,NaN
3,0,0,43,1,2013,1.0,1.0,NaN
4,0,0,51,1,2013,2.0,2.5,NaN
5,0,0,61,1,2013,1.0,1.0,NaN
6,0,0,75,1,2013,1.0,1.0,NaN
7,0,0,88,1,2013,1.0,1.0,NaN
8,0,0,95,1,2013,1.0,1.0,NaN
9,0,0,96,1,2013,1.0,1.0,NaN


In [34]:
df_train = df_train.loc[:,~df_train.columns.duplicated()]

In [35]:
df_train

,date_block_num,shop_id,item_id,month,Year,item_cnt_month,item_cnt_mean_month,item_cnt_prev_month
0,0,0,32,1,2013,6.0,8.000000,NaN
1,0,0,33,1,2013,3.0,3.000000,NaN
2,0,0,35,1,2013,1.0,7.500000,NaN
3,0,0,43,1,2013,1.0,1.000000,NaN
4,0,0,51,1,2013,2.0,2.500000,NaN
...,...,...,...,...,...,...,...,...
1609119,33,59,22087,10,2015,6.0,6.424242,6.0
1609120,33,59,22088,10,2015,2.0,6.187500,2.0
1609121,33,59,22091,10,2015,1.0,2.294118,1.0
1609122,33,59,22100,10,2015,1.0,1.000000,1.0


In [36]:
df_train.fillna(0.0)

,date_block_num,shop_id,item_id,month,Year,item_cnt_month,item_cnt_mean_month,item_cnt_prev_month
0,0,0,32,1,2013,6.0,8.000000,0.0
1,0,0,33,1,2013,3.0,3.000000,0.0
2,0,0,35,1,2013,1.0,7.500000,0.0
3,0,0,43,1,2013,1.0,1.000000,0.0
4,0,0,51,1,2013,2.0,2.500000,0.0
...,...,...,...,...,...,...,...,...
1609119,33,59,22087,10,2015,6.0,6.424242,6.0
1609120,33,59,22088,10,2015,2.0,6.187500,2.0
1609121,33,59,22091,10,2015,1.0,2.294118,1.0
1609122,33,59,22100,10,2015,1.0,1.000000,1.0


In [37]:
df_train = pd.merge (df_train, df_items , how = 'left', on = ['item_id'])

In [38]:
df_train = pd.merge (df_train, df_cats, how = 'left' , on =['item_category_id'])

In [39]:
df_train = pd.merge (df_train, df_shops, how ='left', on = ['shop_id'])

In [40]:
df_train 

,date_block_num,shop_id,item_id,month,Year,item_cnt_month,item_cnt_mean_month,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name
0,0,0,32,1,2013,6.0,8.000000,NaN,1+1,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
1,0,0,33,1,2013,3.0,3.000000,NaN,1+1 (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран"
2,0,0,35,1,2013,1.0,7.500000,NaN,10 ЛЕТ СПУСТЯ,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
3,0,0,43,1,2013,1.0,1.000000,NaN,100 МИЛЛИОНОВ ЕВРО,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
4,0,0,51,1,2013,2.0,2.500000,NaN,100 лучших произведений классики (mp3-CD) (Dig...,57,Музыка - MP3,"!Якутск Орджоникидзе, 56 фран"
...,...,...,...,...,...,...,...,...,...,...,...,...
1609119,33,59,22087,10,2015,6.0,6.424242,6.0,Элемент питания DURACELL LR03-BC2,83,Элементы питания,"Ярославль ТЦ ""Альтаир"""
1609120,33,59,22088,10,2015,2.0,6.187500,2.0,Элемент питания DURACELL LR06-BC2,83,Элементы питания,"Ярославль ТЦ ""Альтаир"""
1609121,33,59,22091,10,2015,1.0,2.294118,1.0,Элемент питания DURACELL TURBO LR 03 2*BL,83,Элементы питания,"Ярославль ТЦ ""Альтаир"""
1609122,33,59,22100,10,2015,1.0,1.000000,1.0,Энциклопедия Adventure Time,42,"Книги - Артбуки, энциклопедии","Ярославль ТЦ ""Альтаир"""


In [41]:
# test dataset adding november month 

In [42]:
# adding november mnth

df_test['month'] = 11
df_test['year'] = 2015
df_test['date_block_num'] = 34

In [43]:
df_test

,ID,shop_id,item_id,month,year,date_block_num
0,0,5,5037,11,2015,34
1,1,5,5320,11,2015,34
2,2,5,5233,11,2015,34
3,3,5,5232,11,2015,34
4,4,5,5268,11,2015,34
...,...,...,...,...,...,...
214195,214195,45,18454,11,2015,34
214196,214196,45,16188,11,2015,34
214197,214197,45,15757,11,2015,34
214198,214198,45,19648,11,2015,34


In [44]:
# add meanfeature

df_test = pd.merge(df_test, shop_item_mean, how = 'left', on = ['shop_id','item_id'])

In [45]:
df_test

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_mean_month
0,0,5,5037,11,2015,34,1.444444
1,1,5,5320,11,2015,34,NaN
2,2,5,5233,11,2015,34,2.000000
3,3,5,5232,11,2015,34,1.000000
4,4,5,5268,11,2015,34,NaN
...,...,...,...,...,...,...,...
214195,214195,45,18454,11,2015,34,1.800000
214196,214196,45,16188,11,2015,34,NaN
214197,214197,45,15757,11,2015,34,1.166667
214198,214198,45,19648,11,2015,34,NaN


In [46]:
df_test.fillna(0.0)

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_mean_month
0,0,5,5037,11,2015,34,1.444444
1,1,5,5320,11,2015,34,0.000000
2,2,5,5233,11,2015,34,2.000000
3,3,5,5232,11,2015,34,1.000000
4,4,5,5268,11,2015,34,0.000000
...,...,...,...,...,...,...,...
214195,214195,45,18454,11,2015,34,1.800000
214196,214196,45,16188,11,2015,34,0.000000
214197,214197,45,15757,11,2015,34,1.166667
214198,214198,45,19648,11,2015,34,0.000000


In [47]:
# add previous month feature

df_test = pd.merge(df_test,shop_prev_month, how = 'left', on = ['shop_id','item_id'])

In [48]:
df_test

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_mean_month,item_cnt_prev_month
0,0,5,5037,11,2015,34,1.444444,NaN
1,1,5,5320,11,2015,34,NaN,NaN
2,2,5,5233,11,2015,34,2.000000,1.0
3,3,5,5232,11,2015,34,1.000000,NaN
4,4,5,5268,11,2015,34,NaN,NaN
...,...,...,...,...,...,...,...,...
214195,214195,45,18454,11,2015,34,1.800000,1.0
214196,214196,45,16188,11,2015,34,NaN,NaN
214197,214197,45,15757,11,2015,34,1.166667,NaN
214198,214198,45,19648,11,2015,34,NaN,NaN


In [49]:
df_test.fillna(0.0)

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_mean_month,item_cnt_prev_month
0,0,5,5037,11,2015,34,1.444444,0.0
1,1,5,5320,11,2015,34,0.000000,0.0
2,2,5,5233,11,2015,34,2.000000,1.0
3,3,5,5232,11,2015,34,1.000000,0.0
4,4,5,5268,11,2015,34,0.000000,0.0
...,...,...,...,...,...,...,...,...
214195,214195,45,18454,11,2015,34,1.800000,1.0
214196,214196,45,16188,11,2015,34,0.000000,0.0
214197,214197,45,15757,11,2015,34,1.166667,0.0
214198,214198,45,19648,11,2015,34,0.000000,0.0


In [50]:

df_test = pd.merge (df_test, df_items , how = 'left', on = ['item_id'])

In [51]:
df_test = pd.merge (df_test, df_cats, how = 'left' , on =['item_category_id'])

In [52]:
df_shops

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


In [53]:
df_test = pd.merge (df_test, df_shops, how ='left', on = ['shop_id'])

In [54]:
df_test

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_mean_month,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name
0,0,5,5037,11,2015,34,1.444444,NaN,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад"""
1,1,5,5320,11,2015,34,NaN,NaN,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,"Вологда ТРЦ ""Мармелад"""
2,2,5,5233,11,2015,34,2.000000,1.0,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад"""
3,3,5,5232,11,2015,34,1.000000,NaN,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,"Вологда ТРЦ ""Мармелад"""
4,4,5,5268,11,2015,34,NaN,NaN,"Need for Speed [PS4, русская версия]",20,Игры - PS4,"Вологда ТРЦ ""Мармелад"""
...,...,...,...,...,...,...,...,...,...,...,...,...
214195,214195,45,18454,11,2015,34,1.800000,1.0,СБ. Союз 55,55,Музыка - CD локального производства,"Самара ТЦ ""ПаркХаус"""
214196,214196,45,16188,11,2015,34,NaN,NaN,Настольная игра Нано Кёрлинг,64,Подарки - Настольные игры,"Самара ТЦ ""ПаркХаус"""
214197,214197,45,15757,11,2015,34,1.166667,NaN,НОВИКОВ АЛЕКСАНДР Новая коллекция,55,Музыка - CD локального производства,"Самара ТЦ ""ПаркХаус"""
214198,214198,45,19648,11,2015,34,NaN,NaN,ТЕРЕМ - ТЕРЕМОК сб.м/ф (Регион),40,Кино - DVD,"Самара ТЦ ""ПаркХаус"""


In [55]:
df_test.shape

(214200, 12)

In [56]:
df_train.shape

(1609124, 12)

In [57]:
df_test['item_cnt_month'] = 0

In [58]:
df_test

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_mean_month,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name,item_cnt_month
0,0,5,5037,11,2015,34,1.444444,NaN,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0
1,1,5,5320,11,2015,34,NaN,NaN,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,"Вологда ТРЦ ""Мармелад""",0
2,2,5,5233,11,2015,34,2.000000,1.0,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0
3,3,5,5232,11,2015,34,1.000000,NaN,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,"Вологда ТРЦ ""Мармелад""",0
4,4,5,5268,11,2015,34,NaN,NaN,"Need for Speed [PS4, русская версия]",20,Игры - PS4,"Вологда ТРЦ ""Мармелад""",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,214195,45,18454,11,2015,34,1.800000,1.0,СБ. Союз 55,55,Музыка - CD локального производства,"Самара ТЦ ""ПаркХаус""",0
214196,214196,45,16188,11,2015,34,NaN,NaN,Настольная игра Нано Кёрлинг,64,Подарки - Настольные игры,"Самара ТЦ ""ПаркХаус""",0
214197,214197,45,15757,11,2015,34,1.166667,NaN,НОВИКОВ АЛЕКСАНДР Новая коллекция,55,Музыка - CD локального производства,"Самара ТЦ ""ПаркХаус""",0
214198,214198,45,19648,11,2015,34,NaN,NaN,ТЕРЕМ - ТЕРЕМОК сб.м/ф (Регион),40,Кино - DVD,"Самара ТЦ ""ПаркХаус""",0


In [59]:
df_test.isna().sum()

ID                          0
shop_id                     0
item_id                     0
month                       0
year                        0
date_block_num              0
item_cnt_mean_month    102796
item_cnt_prev_month    185520
item_name                   0
item_category_id            0
item_category_name          0
shop_name                   0
item_cnt_month              0
dtype: int64

In [60]:
df_test.fillna(0.0)

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_mean_month,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name,item_cnt_month
0,0,5,5037,11,2015,34,1.444444,0.0,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0
1,1,5,5320,11,2015,34,0.000000,0.0,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,"Вологда ТРЦ ""Мармелад""",0
2,2,5,5233,11,2015,34,2.000000,1.0,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0
3,3,5,5232,11,2015,34,1.000000,0.0,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,"Вологда ТРЦ ""Мармелад""",0
4,4,5,5268,11,2015,34,0.000000,0.0,"Need for Speed [PS4, русская версия]",20,Игры - PS4,"Вологда ТРЦ ""Мармелад""",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,214195,45,18454,11,2015,34,1.800000,1.0,СБ. Союз 55,55,Музыка - CD локального производства,"Самара ТЦ ""ПаркХаус""",0
214196,214196,45,16188,11,2015,34,0.000000,0.0,Настольная игра Нано Кёрлинг,64,Подарки - Настольные игры,"Самара ТЦ ""ПаркХаус""",0
214197,214197,45,15757,11,2015,34,1.166667,0.0,НОВИКОВ АЛЕКСАНДР Новая коллекция,55,Музыка - CD локального производства,"Самара ТЦ ""ПаркХаус""",0
214198,214198,45,19648,11,2015,34,0.000000,0.0,ТЕРЕМ - ТЕРЕМОК сб.м/ф (Регион),40,Кино - DVD,"Самара ТЦ ""ПаркХаус""",0


In [61]:
#label encoding

In [62]:
lbl = preprocessing.LabelEncoder()
df_train['item_name'] = lbl.fit_transform(df_train['item_name'])
df_train['item_category_name'] = lbl.fit_transform(df_train['item_category_name'])
df_train['shop_name'] = lbl.fit_transform(df_train['shop_name'])


df_test['item_name'] = lbl.fit_transform(df_test['item_name'])
df_test['item_category_name'] = lbl.fit_transform(df_test['item_category_name'])
df_test['shop_name'] = lbl.fit_transform(df_test['shop_name'])



In [63]:
df_train.isna().sum()

date_block_num               0
shop_id                      0
item_id                      0
month                        0
Year                         0
item_cnt_month               0
item_cnt_mean_month          0
item_cnt_prev_month    1391821
item_name                    0
item_category_id             0
item_category_name           0
shop_name                    0
dtype: int64

In [64]:
df_train = df_train.fillna(0.)

In [65]:
df_train.dtypes

date_block_num           int64
shop_id                  int64
item_id                  int64
month                    int64
Year                     int64
item_cnt_month         float64
item_cnt_mean_month    float64
item_cnt_prev_month    float64
item_name                int64
item_category_id         int64
item_category_name       int64
shop_name                int64
dtype: object

In [66]:
df_test.dtypes

ID                       int64
shop_id                  int64
item_id                  int64
month                    int64
year                     int64
date_block_num           int64
item_cnt_mean_month    float64
item_cnt_prev_month    float64
item_name                int64
item_category_id         int64
item_category_name       int64
shop_name                int64
item_cnt_month           int64
dtype: object

In [67]:
df_train.head()

,date_block_num,shop_id,item_id,month,Year,item_cnt_month,item_cnt_mean_month,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name
0,0,0,32,1,2013,6.0,8.0,0.0,32,40,40,0
1,0,0,33,1,2013,3.0,3.0,0.0,33,37,37,0
2,0,0,35,1,2013,1.0,7.5,0.0,35,40,40,0
3,0,0,43,1,2013,1.0,1.0,0.0,43,40,40,0
4,0,0,51,1,2013,2.0,2.5,0.0,51,57,57,0


In [68]:
df_test.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_mean_month,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name,item_cnt_month
0,0,5,5037,11,2015,34,1.444444,NaN,1195,19,11,3,0
1,1,5,5320,11,2015,34,NaN,NaN,1254,55,39,3,0
2,2,5,5233,11,2015,34,2.000000,1.0,1239,19,11,3,0
3,3,5,5232,11,2015,34,1.000000,NaN,1238,23,15,3,0
4,4,5,5268,11,2015,34,NaN,NaN,1245,20,12,3,0


In [69]:
df_test = df_test.fillna(0.)

In [70]:
df_test.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_mean_month,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name,item_cnt_month
0,0,5,5037,11,2015,34,1.444444,0.0,1195,19,11,3,0
1,1,5,5320,11,2015,34,0.000000,0.0,1254,55,39,3,0
2,2,5,5233,11,2015,34,2.000000,1.0,1239,19,11,3,0
3,3,5,5232,11,2015,34,1.000000,0.0,1238,23,15,3,0
4,4,5,5268,11,2015,34,0.000000,0.0,1245,20,12,3,0


In [71]:
df_test.isna().sum()

ID                     0
shop_id                0
item_id                0
month                  0
year                   0
date_block_num         0
item_cnt_mean_month    0
item_cnt_prev_month    0
item_name              0
item_category_id       0
item_category_name     0
shop_name              0
item_cnt_month         0
dtype: int64

In [72]:
df_train.isna().sum()

date_block_num         0
shop_id                0
item_id                0
month                  0
Year                   0
item_cnt_month         0
item_cnt_mean_month    0
item_cnt_prev_month    0
item_name              0
item_category_id       0
item_category_name     0
shop_name              0
dtype: int64

In [73]:
df_train.dtypes

date_block_num           int64
shop_id                  int64
item_id                  int64
month                    int64
Year                     int64
item_cnt_month         float64
item_cnt_mean_month    float64
item_cnt_prev_month    float64
item_name                int64
item_category_id         int64
item_category_name       int64
shop_name                int64
dtype: object

In [74]:
df_test.dtypes

ID                       int64
shop_id                  int64
item_id                  int64
month                    int64
year                     int64
date_block_num           int64
item_cnt_mean_month    float64
item_cnt_prev_month    float64
item_name                int64
item_category_id         int64
item_category_name       int64
shop_name                int64
item_cnt_month           int64
dtype: object

In [75]:
df_test['date_block_num'].unique()

array([34])

In [76]:
df_test

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_mean_month,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name,item_cnt_month
0,0,5,5037,11,2015,34,1.444444,0.0,1195,19,11,3,0
1,1,5,5320,11,2015,34,0.000000,0.0,1254,55,39,3,0
2,2,5,5233,11,2015,34,2.000000,1.0,1239,19,11,3,0
3,3,5,5232,11,2015,34,1.000000,0.0,1238,23,15,3,0
4,4,5,5268,11,2015,34,0.000000,0.0,1245,20,12,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,214195,45,18454,11,2015,34,1.800000,1.0,4322,55,39,29,0
214196,214196,45,16188,11,2015,34,0.000000,0.0,3902,64,47,29,0
214197,214197,45,15757,11,2015,34,1.166667,0.0,3720,55,39,29,0
214198,214198,45,19648,11,2015,34,0.000000,0.0,4563,40,30,29,0


In [77]:
import time

In [78]:
from xgboost import XGBRegressor

In [79]:
import gc

In [80]:
df_train.columns

Index(['date_block_num', 'shop_id', 'item_id', 'month', 'Year',
       'item_cnt_month', 'item_cnt_mean_month', 'item_cnt_prev_month',
       'item_name', 'item_category_id', 'item_category_name', 'shop_name'],
      dtype='object')

In [81]:
df_train = df_train.rename(columns ={'Year':'year'})

In [82]:
df_train['ID'] = df_train.index

In [83]:
df_train = df_train[['ID', 'shop_id', 'item_id', 'month', 'year', 'date_block_num', 'item_cnt_mean_month', 'item_cnt_prev_month', 'item_name', 'item_category_id', 'item_category_name', 'shop_name','item_cnt_month']]

In [84]:
X_train = df_train[df_train['date_block_num'] < 33].drop(['item_cnt_month'], axis=1)
Y_train = df_train[df_train['date_block_num'] < 33]['item_cnt_month']
X_valid = df_train[df_train['date_block_num'] == 33].drop(['item_cnt_month'], axis=1)
Y_valid = df_train[df_train['date_block_num'] == 33]['item_cnt_month']
X_test = df_test.drop(['item_cnt_month'], axis=1)


gc.collect();

In [85]:
ts = time.time()

model = XGBRegressor(
    max_depth=7,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    gamma = 0.005,
    eta=0.1,    
    seed=42)

model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=10, 
    early_stopping_rounds = 40,
    )

time.time() - ts

[0]	validation_0-rmse:8.22063	validation_1-rmse:14.15497
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 40 rounds.
[10]	validation_0-rmse:6.31227	validation_1-rmse:12.18561
[20]	validation_0-rmse:5.89361	validation_1-rmse:11.85101
[30]	validation_0-rmse:5.78352	validation_1-rmse:11.77247
[40]	validation_0-rmse:5.71585	validation_1-rmse:11.77893
[50]	validation_0-rmse:5.66708	validation_1-rmse:11.78607
[60]	validation_0-rmse:5.63540	validation_1-rmse:11.82246
[70]	validation_0-rmse:5.62731	validation_1-rmse:11.84384
Stopping. Best iteration:
[37]	validation_0-rmse:5.74072	validation_1-rmse:11.76039



102.62513709068298

In [86]:
X_train.dtypes

ID                       int64
shop_id                  int64
item_id                  int64
month                    int64
year                     int64
date_block_num           int64
item_cnt_mean_month    float64
item_cnt_prev_month    float64
item_name                int64
item_category_id         int64
item_category_name       int64
shop_name                int64
dtype: object

In [87]:
Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)


X_train_level2 = pd.DataFrame({
    "ID": np.arange(Y_pred.shape[0]), 
    "item_cnt_month": Y_pred
})
X_train_level2.to_csv('xgb_valid.csv', index=False)

submission = pd.DataFrame({
    "ID": np.arange(Y_test.shape[0]), 
    "item_cnt_month": Y_test
})
submission.to_csv('xgb_submission.csv', index=False)

In [88]:
ls

Untitled.ipynb         sales_train.csv*       xgb_submission.csv
Untitled1.ipynb        sample_submission.csv* xgb_valid.csv
item_categories.csv*   shops.csv*
items.csv*             test.csv*


In [90]:
pd.read_csv('xgb_submission.csv')

,ID,item_cnt_month
0,0,1.561273
1,1,0.709413
2,2,3.071988
3,3,1.170880
4,4,1.123471
...,...,...
214195,214195,1.832663
214196,214196,0.720309
214197,214197,1.130458
214198,214198,0.698680


In [91]:
pd.read_csv('xgb_valid.csv')

,ID,item_cnt_month
0,0,1.178601
1,1,1.885421
2,2,1.005129
3,3,1.059687
4,4,0.989628
...,...,...
31526,31526,5.417258
31527,31527,6.186431
31528,31528,4.318553
31529,31529,0.840913
